# README

This readme file contains documentation for combining and processing a dataset related to Wikipedia page traffic and analyzing it.


Below are the steps involved
## Gather the data
We need to collect monthly data from the below
There are 5 things here
Legacy Pagecounts API
1. Desktop data
2. Mobile data

Pageviews API
1. Desktop data
2. Mobile web data
3. Mobile app traffic data

### Imports
Before we do any processing, we need to import all required libraries for python. These are available on PyPi repository

In [357]:
import json
import requests
import pandas as pd
from datetime import datetime
import plotly.express as px


### Constants and common definitions
The below code will declare constants like endpoints, headers and define the access points supported by the different endpoints

In [70]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/en.wikipedia.org/{access}/monthly/{start}/{end}'
endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/en.wikipedia.org/{access}/{agent}/monthly/{start}/{end}'
headers = {
    'User-Agent': 'https://github.com/vikrantb',
    'From': 'vikrantb@uw.edu'
}

legacy_access_points = ['desktop-site', 'mobile-site']
page_view_access_points = ['desktop', 'mobile-app', 'mobile-web']

### Common helper functions

Below are some helper functions that will be used throughout. With an explanation

#### api_call
This method makes a call to the endpoint provided with the parameters provided. It also has an option to directly extract the items

#### get_wiki_df
This method returns the pagecount/pageviews information in a dataframe format

#### get_from_legacy
This method is a wrapper method on top of api_call specifically for legacy endpoint

#### get_from_page_views
This method is a wrapper method on top of api_call specifically for pageviews endpoint


In [284]:
def api_call(endpoint,parameters, extract_items=False):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response['items'] if extract_items else response

def get_from_legacy(access, start, end, save_file=False):
    params = {"project" : "en.wikipedia.org",
             "access" : access,
             "granularity" : "monthly",
             "start" : "{start}0100".format(start=start),
             "end" : "{end}0100".format(end=end)
            }
    data = api_call(endpoint=endpoint_legacy, parameters=params)
    file_name="pagecounts_{access}_{start}-{end}.json".format(access=access, start=start, end=end)
    with open(file_name, 'w') as f:
        json.dump(data, f)


def get_from_page_views(access, start, end, save_file=False):
    params = {"project" : "en.wikipedia.org",
             "access" : access,
             "agent" : "user",
             "granularity" : "monthly",
             "start" : "{start}0100".format(start=start),
             "end" : "{end}0100".format(end=end)
            }
    data = api_call(endpoint=endpoint_pageviews, parameters=params)
    file_name="pageviews_{access}_{start}-{end}.json".format(access=access, start=start, end=end)
    with open(file_name, 'w') as f:
        json.dump(data, f)

def get_wiki_df(file_path):
    wiki_json = ''
    with open(file_path) as f:
        wiki_json = str(json.load(f)['items']).replace("\'", "\"")
    df = pd.read_json(wiki_json, orient='records', convert_dates=False)
    df['timestamp'] = df['timestamp'].apply(str)
    df['year'] = df['timestamp'].apply(lambda x: x[:4])
    df['month'] = df['timestamp'].apply(lambda x: x[4:6])
    df['num_views'] = df['views'] if 'views' in df.columns else df['count']
    return df[['year', 'month', 'num_views']]


## Step 1: Gathering the data

In this step we will gather the wikipedia pagecount data per month level.
As noted below, the data is available in legacy endpoint from January 2008 to July 2016
As noted below, the data is available in pageviews endpoint from July 2015 2008 to September 2020.

There is an overlap for data from July 2015 to July 2016

After execution of the below step, there will be 5 json files created as below

- pagecounts_desktop-site_200801-201607.json - desktop site information from legacy api from January 2008 to July 2016
- pagecounts_mobile-site_200801-201607.json - mobile site information from legacy api from January 2008 to July 2016
- pageviews_desktop_201507-202009.json - desktop information from pageviews api from July 2015 to September 2020
- pageviews_mobile-app_201507-202009.json - mobile app information from pageviews api from July 2015 to September 2020
- pageviews_mobile-web_201507-202009.json - mobile web information from pageviews api from July 2015 to September 2020

In [285]:
legacy_start_date='200801'
legacy_end_date='201607'
for access_type in legacy_access_points:
    get_from_legacy(access=access_type, start=legacy_start_date,end=legacy_end_date, save_file=True)

legacy_start_date='201507'
legacy_end_date='202009'
for access_type in page_view_access_points:
    get_from_page_views(access=access_type, start=legacy_start_date,end=legacy_end_date, save_file=True)

## Step 2: Processing the data
The objective of this step is to combine data from all the json files collected in the gathering step into a single csv file upon which we will run analysis.

For the pageviews api, we will sum the counts from mobile-app and mobile-web to create a single value for mobile traffic.

For all the data, we will be only using the `YYYYMM` part of the timestamp and ignoring the `DDHH` part

In [303]:
pagecounts_desktop_fp = 'pagecounts_desktop-site_200801-201607.json'
pagecounts_mobile_fp = 'pagecounts_mobile-site_200801-201607.json'
pageviews_desktop_fp = 'pageviews_desktop_201507-202009.json'
pageviews_mobile_app_fp = 'pageviews_mobile-app_201507-202009.json'
pageviews_mobile_web_fp = 'pageviews_mobile-web_201507-202009.json'

pagecounts_desktop_df = get_wiki_df(pagecounts_desktop_fp)
pagecounts_mobile_df = get_wiki_df(pagecounts_mobile_fp)
pageviews_desktop_df = get_wiki_df(pageviews_desktop_fp)
pageviews_mobile_app_df = get_wiki_df(pageviews_mobile_app_fp)
pageviews_mobile_web_df = get_wiki_df(pageviews_mobile_web_fp)
pageviews_mobile_df = pd.concat([pageviews_mobile_app_df,pageviews_mobile_web_df])
pageviews_mobile_df = pageviews_mobile_df.groupby(['year', 'month']).sum().reset_index()

pagecount_all_df = pd.concat([pagecounts_desktop_df, pagecounts_mobile_df])
pagecount_all_df = pagecount_all_df.groupby(['year', 'month']).sum().reset_index()

pageview_all_df = pd.concat([pageviews_desktop_df, pageviews_mobile_df])
pageview_all_df = pageview_all_df.groupby(['year', 'month']).sum().reset_index()


In [345]:
all_of_all_df = pd.concat([pagecount_all_df, pageview_all_df])[['year', 'month']]
all_years = all_of_all_df.groupby(['year', 'month']).sum().reset_index()

In [346]:
all_years['num_views'] = 0
all_years.head(5)

,year,month,num_views
0,2008,01,0
1,2008,02,0
2,2008,03,0
3,2008,04,0
4,2008,05,0


In [347]:
def add_df(all_years, subset_df, col_name):
    all_years = pd.merge(all_years, subset_df, how='left', on=['year', 'month'])
    all_years[col_name] = all_years['num_views_y']
    all_years['num_views'] = 0
    all_years = all_years.drop(columns=['num_views_x', 'num_views_y'])
    return all_years

In [365]:
all_years = add_df(all_years, pagecount_all_df, 'pagecount_all_views')
all_years = add_df(all_years, pagecounts_desktop_df, 'pagecount_desktop_views')
all_years = add_df(all_years, pagecounts_mobile_df, 'pagecount_mobile_views')
all_years = add_df(all_years, pageview_all_df, 'pageview_all_views')
all_years = add_df(all_years, pageviews_desktop_df, 'pageview_desktop_views')
all_years = add_df(all_years, pageviews_mobile_df, 'pageview_mobile_views')
all_years = all_years.drop(columns=['num_views'])

In [367]:
all_years = all_years.fillna(0)

In [377]:
all_years.to_csv('en-wikipedia_traffic_200712-202008.csv')

In [378]:
all_years.columns

Index(['year', 'month', 'pagecount_all_views', 'pagecount_desktop_views',
       'pagecount_mobile_views', 'pageview_all_views',
       'pageview_desktop_views', 'pageview_mobile_views'],
      dtype='object')

In [383]:
all_years['date'] = all_years['year'] + '-' + all_years['month'] + '01'
all_years.head()

,year,month,pagecount_all_views,pagecount_desktop_views,pagecount_mobile_views,pageview_all_views,pageview_desktop_views,pageview_mobile_views,date
0,2008,01,4.930903e+09,4.930903e+09,0.0,0.0,0.0,0.0,2008-0101
1,2008,02,4.818394e+09,4.818394e+09,0.0,0.0,0.0,0.0,2008-0201
2,2008,03,4.955406e+09,4.955406e+09,0.0,0.0,0.0,0.0,2008-0301
3,2008,04,5.159162e+09,5.159162e+09,0.0,0.0,0.0,0.0,2008-0401
4,2008,05,5.584691e+09,5.584691e+09,0.0,0.0,0.0,0.0,2008-0501


In [384]:
fig = px.line(all_years, x="date", y=['pagecount_all_views', 'pagecount_desktop_views',
       'pagecount_mobile_views', 'pageview_all_views',
       'pageview_desktop_views', 'pageview_mobile_views'],
              title='custom tick labels with ticklabelmode="period"')
fig.show()